<a href="https://www.kaggle.com/code/hari141v/semantic-search-evaluation-elasticsearch?scriptVersionId=187998549" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### About 
### Information retrieval using elasticsearch and custom search methods. Calculate recall, mrr from search results
### The code is copied from [LLM_ZOOMCAMP](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-vector-search/eval/evaluate-text.ipynb)

### Import necessary libraries and packages

In [1]:
pip install elasticsearch -qq 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentence_transformers==2.7.0 -qq

Note: you may need to restart the kernel to use updated packages.


In [3]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
!tar -xzf elasticsearch-8.14.0-linux-x86_64.tar.gz
!chown -R root:root elasticsearch-8.14.0
!adduser --disabled-password --gecos '' elasticsearch_user
!chown -R elasticsearch_user:elasticsearch_user elasticsearch-8.14.0

--2024-07-12 17:01:45--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588482330 (561M) [application/x-gzip]
Saving to: 'elasticsearch-8.14.0-linux-x86_64.tar.gz'

elasticsearch-8.14. 100%[===================>] 561.22M  8.47MB/s    in 76s     

2024-07-12 17:03:02 (7.38 MB/s) - 'elasticsearch-8.14.0-linux-x86_64.tar.gz' saved [588482330/588482330]

Adding user `elasticsearch_user' ...
Adding new group `elasticsearch_user' (1000) ...
Adding new user `elasticsearch_user' (1001) with group `elasticsearch_user' ...
Creating home directory `/home/elasticsearch_user' ...
Copying files from `/etc/skel' ...


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import io
import os
import json
import requests
from tqdm.notebook import tqdm
import subprocess
import time
import json
import elasticsearch
from elasticsearch import Elasticsearch

### Elasticsearch setup

In [5]:
# Run elasticsearch
command = "sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(20)

In [6]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

root         130      33  0 17:03 ?        00:00:00 /bin/sh -c sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
root         131     130  0 17:03 ?        00:00:00 sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
elastic+     132     131 34 17:03 ?        00:00:06 /kaggle/working/elasticsearch-8.14.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=./elasticsearch-8.14.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/kaggle/working/elasticsearch-8.14.0 -Des.path.conf=/kaggle/working/elasticsearch-8.14.0/config -Des.distribution.type=tar -cp /kaggle/working/elasticsearch-8.14.0/lib/*:/kaggle/working/elasticsearch-8.14.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher -E xpack.security.enabled=false -E thread_pool.se

In [7]:
es_client = Elasticsearch('http://localhost:9200/', request_timeout=60) 

In [9]:
!curl localhost:9200

{
  "name" : "911ce8282b93",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "ph4BbJK7TP-kcpk2PHM3jQ",
  "version" : {
    "number" : "8.14.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "8d96bbe3bf5fed931f3119733895458eab75dca9",
    "build_date" : "2024-06-03T10:05:49.073003402Z",
    "build_snapshot" : false,
    "lucene_version" : "9.10.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


### Get document file from coure repo

In [10]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json

--2024-07-12 17:24:31--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 699257 (683K) [text/plain]
Saving to: 'documents-with-ids.json'

documents-with-ids. 100%[===================>] 682.87K  3.76MB/s    in 0.2s    

2024-07-12 17:24:31 (3.76 MB/s) - 'documents-with-ids.json' saved [699257/699257]



### Load the document

In [11]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [33]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

### Create and add index to elasticsearch

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Elasticsearch query

In [14]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

### Sample search

In [15]:
elastic_search(
    query="I just discovered the course. Can I still join?",
    course="data-engineering-zoomcamp"
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp',
  'id': '63394d91'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it fin

### Get ground truth data(LLM generated question data) from course repo

In [16]:
df_ground_truth_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/ground-truth-data.csv'
df_ground_truth=pd.read_csv(df_ground_truth_url)
df_ground_truth.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


### Convert each row to dictionary

In [17]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [18]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

### Search the question and match relevance based on the doc_id

In [26]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4627 [00:00<?, ?it/s]

### Sample relevance result

In [20]:
example = [
    [True, False, False, False, False], # 1, 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1 
    [False, False, True, False, False],  # 1/3
    [False, False, False, False, False], # 0
]

# 1 => 1
# 2 => 1 / 2 = 0.5
# 3 => 1 / 3 = 0.3333
# 4 => 0.25
# 5 => 0.2
# rank => 1 / rank
# none => 0

### Hit-rate(True positive rate, Recall, Senstivity) Calculation

```
[True, False, False, False, False], # 1, 
[False, False, False, False, False], # 0
[False, False, False, False, False], # 0 
[False, False, False, False, False], # 0
[False, False, False, False, False], # 0 
[True, False, False, False, False], # 1
[True, False, False, False, False], # 1
[True, False, False, False, False], # 1
[True, False, False, False, False], # 1
[True, False, False, False, False], # 1 
[False, False, True, False, False],  # 1
[False, False, False, False, False] # 0

Total_1 = 7
Total_sample = 12
Hit-rate(True positive rate, Recall, Sensitivity) = TP/TP+FN = 7/12 = 0.5833333333333334

```

In [21]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

### Mean Reciprocal Rank (MRR) calculation
```
True and position(rank) of true = reciprocal rank

[True, False, False, False, False], # 1/1
[False, False, False, False, False], # 0
[False, False, False, False, False], # 0 
[False, False, False, False, False], # 0
[False, False, False, False, False], # 0 
[True, False, False, False, False], # 1/1
[True, False, False, False, False], # 1/1
[True, False, False, False, False], # 1/1
[True, False, False, False, False], # 1/1
[True, False, False, False, False], # 1/1
[False, False, True, False, False],  # 1/3
[False, False, False, False, False] # 0

Total_1_rank = 1/1 + 1/1 + 1/1 + 1/1 + 1/1 +1/1 + 1/3 = 6.333333333333333
Total_sample = 12
Mean Reciprocal Rank (MRR) = rank/total_sample = 6.333333333333333/12 = 0.5277777777777778

```

In [22]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [23]:
hit_rate(example)

0.5833333333333334

In [24]:
mrr(example)

0.5277777777777778

### Hit-rate and MRR for entire document

In [27]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7395720769397017, 0.6030149124702835)

### Get minisearch script from course repo

In [29]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-12 17:30:51--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-12 17:30:51 (44.4 MB/s) - 'minsearch.py' saved [3832/3832]



### Create TF-IDF

In [30]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

### Cosine similarity search based on the TF-IDF

In [31]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

### Search relevance document using custom search

In [32]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4627 [00:00<?, ?it/s]

### Result

In [33]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7722066133563864, 0.661454506159499)

Compare with ES results:
```
(0.7395720769397017, 0.6032418413658963)
```

### Funtion to return hit-rate and mrr

In [34]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

### Elastissearch relevance results 

In [35]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.7395720769397017, 'mrr': 0.6030149124702835}

### Custom search relevance results 

In [36]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.7722066133563864, 'mrr': 0.661454506159499}